In [1]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import json
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.request import urlopen
from html_table_parser import parser_functions as parser

# 예금

In [6]:
Deposit_url = "https://new-m.pay.naver.com/savings/list/deposit"

driver_options = webdriver.ChromeOptions()
driver_options.add_argument("headless")

driver = webdriver.Chrome(options=driver_options)
# driver = webdriver.Chrome()
driver.get(Deposit_url)

time.sleep(2)

# 조건 리셋
reset_css = "#content > div.ProductFilterSection_article__0nJDV > div.ProductFilterTags_article__LS1Oc > button > svg"
driver.find_element(By.CSS_SELECTOR, reset_css).click()  

# 1금융권
bank_css = "#content > div.Navigation_article__t42yM > div > ul > li:nth-child(1) > label"
driver.find_element(By.CSS_SELECTOR, bank_css).click()  

# 총 합쳐지는 df 구하기
all_df_deposit = pd.DataFrame()

In [7]:
# 한 페이지에 20개씩 존재
product_class_css = "ProductList_item__QXNrf"
product_list = driver.find_elements(By.CLASS_NAME, product_class_css)


num_page = 1
while True:
    product_class_css = "ProductList_item__QXNrf"
    product_list = driver.find_elements(By.CLASS_NAME, product_class_css)

    for product in product_list:
        href = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")

        # 상품별 새로운 창 만들기
        script = f"window.open('{href}');"
        driver.execute_script(script)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)

        # 상품명 먼저 확인
        product_name_css = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > h3"
        product_name = driver.find_element(By.CSS_SELECTOR, product_name_css).text

        # 표 불러들이기 
        result = urlopen(href)
        html = result.read()
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        

        # 표 에러 확인 및 표 -> df로 변경
        try:
            p = parser.make2d(temp[0])
            table_df=pd.DataFrame(p[1:],columns=p[0])
            len_table_df = len(table_df)
                    # 컬럼 이름 변경해주기 (합쳐주기)
            if table_df.columns[0] == table_df.iloc[0,0]:
                col_name_list = [table_df.columns[0]]
                for e,i in enumerate(list(table_df.loc[0,:])):
                    if e == 0:
                        continue
                    col_name_list.append(table_df.columns[e]+"_"+i)

                table_df.columns= col_name_list
                table_df.drop(index = 0, inplace= True)
                table_df.reset_index(drop= True, inplace= True)
        except:
            table_df = pd.DataFrame()
            len_table_df = 1
        

        # tmp_df 생성
        tmp_df = pd.DataFrame()

        
        # 상품명
        tmp_df['상품명'] = [product_name]*len_table_df

        # 은행명
        bank_css  = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > span.MainInfo_text__V_Fkd"
        bank = driver.find_element(By.CSS_SELECTOR, bank_css).text
        tmp_df['은행명']  = [bank] * len_table_df

        # 금리
        interest_css = "MainInfo_rate__em52j"
        interest_list = driver.find_elements(By.CLASS_NAME, interest_css)

        # 최고 우대금리
        high_interest = interest_list[0].text.split()[-1]

        # 기본 금리
        interest = interest_list[1].text.split()[-1]

        # 보여지는 기본, 최고 우대금리
        tmp_df['기본금리_'] = [interest] * len_table_df
        tmp_df['최고우대금리_'] = [high_interest]*len_table_df

    # 상품유형
        Tag_css = "TagList_tag__xQBbK"
        Tag_list = driver.find_elements(By.CLASS_NAME, Tag_css)
        Tag = ",".join([css.text for css in Tag_list])
        tmp_df['상품유형'] = [Tag] * len_table_df

        # 더보기
        more_css = "#PRODUCT_GUIDE > div > button"
        try:
            more_btn = driver.find_element(By.CSS_SELECTOR, more_css)
            more_btn.click()
        except:
            pass
        
        time.sleep(1)

        # item_info 일단 모으기
        item_info_name_css = "TextList_label__MxF08"
        item_info_name_list = driver.find_elements(By.CLASS_NAME,item_info_name_css )

        item_info_detail_css = "TextList_description__xhuFz"
        item_info_detail_list = driver.find_elements(By.CLASS_NAME, item_info_detail_css)

        for i in range(len(item_info_name_list)):
            if item_info_name_list[i].text =='기간':
                tmp_df['기간_'] = [item_info_detail_list[i].text]* len_table_df
                continue
            tmp_df[item_info_name_list[i].text] = [item_info_detail_list[i].text]* len_table_df
        
        
        # table 합치기
        if len(table_df) != 0:
            tmp_df = pd.concat([tmp_df, table_df], axis = 1)

        #만들어진 창을 닫고 프레임 전환
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        
        try:
            all_df_deposit = pd.concat([all_df_deposit, tmp_df], axis = 0)
        except:
            all_df_deposit.to_csv("./tmp_saving.csv")
            #페이지
    next_page_css = "#content > div.ProductListSection_article__kDyT3 > div.Pagination_article__Rg8y3 > div > button.Pagination_button__ukte8.Pagination_next__2yfET"

    try:
        next_page_btn = driver.find_element(By.CSS_SELECTOR, next_page_css)
        next_page_btn.send_keys()
        next_page_btn.click()
        print(num_page)
        num_page += 1
        time.sleep(1)
    except:
        all_df_deposit.to_csv("./Deposit_bank.csv", encoding = "utf-8-sig")
        driver.close()
        break

1
2
3
4
5
6
7


# 적금

In [ ]:
saving_url = "https://new-m.pay.naver.com/savings/list/saving"

driver_options = webdriver.ChromeOptions()
driver_options.add_argument("headless")

driver = webdriver.Chrome(options=driver_options)
# driver = webdriver.Chrome()
driver.get(saving_url)

time.sleep(2)

# 조건 리셋
reset_css = "#content > div.ProductFilterSection_article__0nJDV > div.ProductFilterTags_article__LS1Oc > button > svg"
driver.find_element(By.CSS_SELECTOR, reset_css).click()  

# 총 합쳐지는 df 구하기
all_df_saving = pd.DataFrame()

In [ ]:
# 한 페이지에 20개씩 존재
product_class_css = "ProductList_item__QXNrf"
product_list = driver.find_elements(By.CLASS_NAME, product_class_css)


num_page = 1
while True:
    product_class_css = "ProductList_item__QXNrf"
    product_list = driver.find_elements(By.CLASS_NAME, product_class_css)

    for product in product_list:
        href = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")

        # 상품별 새로운 창 만들기
        script = f"window.open('{href}');"
        driver.execute_script(script)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)

        # 상품명 먼저 확인
        product_name_css = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > h3"
        product_name = driver.find_element(By.CSS_SELECTOR, product_name_css).text

        # 표 불러들이기 
        result = urlopen(href)
        html = result.read()
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        

        # 표 에러 확인 및 표 -> df로 변경
        try:
            p = parser.make2d(temp[0])
            table_df=pd.DataFrame(p[1:],columns=p[0])
            len_table_df = len(table_df)
                    # 컬럼 이름 변경해주기 (합쳐주기)
            if table_df.columns[0] == table_df.iloc[0,0]:
                col_name_list = [table_df.columns[0]]
                for e,i in enumerate(list(table_df.loc[0,:])):
                    if e == 0:
                        continue
                    col_name_list.append(table_df.columns[e]+"_"+i)

                table_df.columns= col_name_list
                table_df.drop(index = 0, inplace= True)
                table_df.reset_index(drop= True, inplace= True)
        except:
            table_df = pd.DataFrame()
            len_table_df = 1
        

        # tmp_df 생성
        tmp_df = pd.DataFrame()

        
        # 상품명
        tmp_df['상품명'] = [product_name]*len_table_df

        # 은행명
        bank_css  = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > span.MainInfo_text__V_Fkd"
        bank = driver.find_element(By.CSS_SELECTOR, bank_css).text
        tmp_df['은행명']  = [bank] * len_table_df

        # 금리
        interest_css = "MainInfo_rate__em52j"
        interest_list = driver.find_elements(By.CLASS_NAME, interest_css)

        # 최고 우대금리
        high_interest = interest_list[0].text.split()[-1]

        # 기본 금리
        interest = interest_list[1].text.split()[-1]

        # 보여지는 기본, 최고 우대금리
        tmp_df['기본금리_'] = [interest] * len_table_df
        tmp_df['최고우대금리_'] = [high_interest]*len_table_df

    # 상품유형
        Tag_css = "TagList_tag__xQBbK"
        Tag_list = driver.find_elements(By.CLASS_NAME, Tag_css)
        Tag = ",".join([css.text for css in Tag_list])
        tmp_df['상품유형'] = [Tag] * len_table_df

        # 더보기
        more_css = "#PRODUCT_GUIDE > div > button"
        try:
            more_btn = driver.find_element(By.CSS_SELECTOR, more_css)
            more_btn.click()
        except:
            pass
        
        time.sleep(1)

        # item_info 일단 모으기
        item_info_name_css = "TextList_label__MxF08"
        item_info_name_list = driver.find_elements(By.CLASS_NAME,item_info_name_css )

        item_info_detail_css = "TextList_description__xhuFz"
        item_info_detail_list = driver.find_elements(By.CLASS_NAME, item_info_detail_css)

        for i in range(len(item_info_name_list)):
            if item_info_name_list[i].text =='기간':
                tmp_df['기간_'] = [item_info_detail_list[i].text]* len_table_df
                continue
            tmp_df[item_info_name_list[i].text] = [item_info_detail_list[i].text]* len_table_df
        
        
        # table 합치기
        if len(table_df) != 0:
            tmp_df = pd.concat([tmp_df, table_df], axis = 1)

        #만들어진 창을 닫고 프레임 전환
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        
        try:
            all_df_saving = pd.concat([all_df_saving, tmp_df], axis = 0)
        except:
            all_df_saving.to_csv("./tmp_saving.csv")
            #페이지
    next_page_css = "#content > div.ProductListSection_article__kDyT3 > div.Pagination_article__Rg8y3 > div > button.Pagination_button__ukte8.Pagination_next__2yfET"

    try:
        next_page_btn = driver.find_element(By.CSS_SELECTOR, next_page_css)
        next_page_btn.send_keys()
        next_page_btn.click()
        print(num_page)
        num_page += 1
        time.sleep(1)
    except:
        all_df_saving.to_csv("./Saving.csv")
        driver.close()
        break
    

# 파킹통장

In [3]:
parking_url = "https://new-m.pay.naver.com/savings/list/parking"

driver_options = webdriver.ChromeOptions()
driver_options.add_argument("headless")

driver = webdriver.Chrome(options=driver_options)
# driver = webdriver.Chrome()
driver.get(parking_url)

time.sleep(2)

# 조건 리셋
reset_css = "#content > div.ProductFilterSection_article__0nJDV > div.ProductFilterTags_article__LS1Oc > button > svg"
driver.find_element(By.CSS_SELECTOR, reset_css).click()  

# 총 합쳐지는 df 구하기
all_df_parking = pd.DataFrame()

In [4]:
# 한 페이지에 20개씩 존재
product_class_css = "ProductList_item__QXNrf"
product_list = driver.find_elements(By.CLASS_NAME, product_class_css)


num_page = 1
while True:
    product_class_css = "ProductList_item__QXNrf"
    product_list = driver.find_elements(By.CLASS_NAME, product_class_css)

    for product in product_list:
        href = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")

        # 상품별 새로운 창 만들기
        script = f"window.open('{href}');"
        driver.execute_script(script)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)

        # 상품명 먼저 확인
        product_name_css = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > h3"
        product_name = driver.find_element(By.CSS_SELECTOR, product_name_css).text

        # 표 불러들이기 
        result = urlopen(href)
        html = result.read()
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        

        # 표 에러 확인 및 표 -> df로 변경
        try:
            p = parser.make2d(temp[0])
            table_df=pd.DataFrame(p[1:],columns=p[0])
            len_table_df = len(table_df)
                    # 컬럼 이름 변경해주기 (합쳐주기)
            if table_df.columns[0] == table_df.iloc[0,0]:
                col_name_list = [table_df.columns[0]]
                for e,i in enumerate(list(table_df.loc[0,:])):
                    if e == 0:
                        continue
                    col_name_list.append(table_df.columns[e]+"_"+i)

                table_df.columns= col_name_list
                table_df.drop(index = 0, inplace= True)
                table_df.reset_index(drop= True, inplace= True)
        except:
            table_df = pd.DataFrame()
            len_table_df = 1
        

        # tmp_df 생성
        tmp_df = pd.DataFrame()

        
        # 상품명
        tmp_df['상품명'] = [product_name]*len_table_df

        # 은행명
        bank_css  = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > span.MainInfo_text__V_Fkd"
        bank = driver.find_element(By.CSS_SELECTOR, bank_css).text
        tmp_df['은행명']  = [bank] * len_table_df

        # 금리
        interest_css = "MainInfo_rate__em52j"
        interest_list = driver.find_elements(By.CLASS_NAME, interest_css)

        # 최고 우대금리
        high_interest = interest_list[0].text.split()[-1]

        # 최저 금리
        interest = interest_list[1].text.split()[-1]

        # 보여지는 기본, 최저 우대금리
        tmp_df['기본금리_'] = [interest] * len_table_df
        tmp_df['최저우대금리_'] = [high_interest]*len_table_df

    # 상품유형
        Tag_css = "TagList_tag__xQBbK"
        Tag_list = driver.find_elements(By.CLASS_NAME, Tag_css)
        Tag = ",".join([css.text for css in Tag_list])
        tmp_df['상품유형'] = [Tag] * len_table_df

        # 더보기
        more_css = "#PRODUCT_GUIDE > div > button"
        try:
            more_btn = driver.find_element(By.CSS_SELECTOR, more_css)
            more_btn.click()
        except:
            pass
        
        time.sleep(1)

        # item_info 일단 모으기
        item_info_name_css = "TextList_label__MxF08"
        item_info_name_list = driver.find_elements(By.CLASS_NAME,item_info_name_css )

        item_info_detail_css = "TextList_description__xhuFz"
        item_info_detail_list = driver.find_elements(By.CLASS_NAME, item_info_detail_css)

        for i in range(len(item_info_name_list)):
            if item_info_name_list[i].text =='기간':
                tmp_df['기간_'] = [item_info_detail_list[i].text]* len_table_df
                continue
            tmp_df[item_info_name_list[i].text] = [item_info_detail_list[i].text]* len_table_df
        
        
        # table 합치기
        if len(table_df) != 0:
            tmp_df = pd.concat([tmp_df, table_df], axis = 1)

        #만들어진 창을 닫고 프레임 전환
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        
        try:
            all_df_parking = pd.concat([all_df_parking, tmp_df], axis = 0)
        except:
            all_df_parking.to_csv("./tmp_saving.csv")
            #페이지
    next_page_css = "#content > div.ProductListSection_article__kDyT3 > div.Pagination_article__Rg8y3 > div > button.Pagination_button__ukte8.Pagination_next__2yfET"

    try:
        next_page_btn = driver.find_element(By.CSS_SELECTOR, next_page_css)
        next_page_btn.send_keys()
        next_page_btn.click()
        print(num_page)
        num_page += 1
        time.sleep(1)
    except:
        all_df_parking.to_csv("./parking.csv", encoding= 'utf-8-sig')
        driver.close()
        break
    

1
2
3
4
5
6


In [6]:
all_df_parking.columns

Index(['상품명', '은행명', '기본금리_', '최저우대금리_', '상품유형', '금액', '가입방법', '대상', '이자지급',
       '유의', '예금자\n보호', '유형', '구분', '금리', '조건별', '우대조건', '', '기본금리'],
      dtype='object')

# CMA

In [3]:
cma_url = "https://new-m.pay.naver.com/savings/list/cma"

driver_options = webdriver.ChromeOptions()
driver_options.add_argument("headless")

driver = webdriver.Chrome(options=driver_options)
# driver = webdriver.Chrome()
driver.get(cma_url)

time.sleep(2)

# # 조건 리셋
# reset_css = "#content > div.ProductFilterSection_article__0nJDV > div.ProductFilterTags_article__LS1Oc > button > svg"
# driver.find_element(By.CSS_SELECTOR, reset_css).click()  

# 총 합쳐지는 df 구하기
all_df_cma = pd.DataFrame()

In [4]:
# 한 페이지에 20개씩 존재
product_class_css = "ProductList_item__QXNrf"
product_list = driver.find_elements(By.CLASS_NAME, product_class_css)


num_page = 1
while True:
    product_class_css = "ProductList_item__QXNrf"
    product_list = driver.find_elements(By.CLASS_NAME, product_class_css)

    for product in product_list:
        href = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")

        # 상품별 새로운 창 만들기
        script = f"window.open('{href}');"
        driver.execute_script(script)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)

        # 상품명 먼저 확인
        product_name_css = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > h3"
        product_name = driver.find_element(By.CSS_SELECTOR, product_name_css).text

        # 표 불러들이기 
        result = urlopen(href)
        html = result.read()
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        

        # 표 에러 확인 및 표 -> df로 변경
        try:
            p = parser.make2d(temp[0])
            table_df=pd.DataFrame(p[1:],columns=p[0])
            len_table_df = len(table_df)
                    # 컬럼 이름 변경해주기 (합쳐주기)
            if table_df.columns[0] == table_df.iloc[0,0]:
                col_name_list = [table_df.columns[0]]
                for e,i in enumerate(list(table_df.loc[0,:])):
                    if e == 0:
                        continue
                    col_name_list.append(table_df.columns[e]+"_"+i)

                table_df.columns= col_name_list
                table_df.drop(index = 0, inplace= True)
                table_df.reset_index(drop= True, inplace= True)
        except:
            table_df = pd.DataFrame()
            len_table_df = 1
        

        # tmp_df 생성
        tmp_df = pd.DataFrame()

        
        # 상품명
        tmp_df['상품명'] = [product_name]*len_table_df

        # 은행명
        bank_css  = "#content > div.MainInfo_article__iQqCh > div.MainInfo_area-title__hzVI3 > span.MainInfo_text__V_Fkd"
        bank = driver.find_element(By.CSS_SELECTOR, bank_css).text
        tmp_df['은행명']  = [bank] * len_table_df

        # 금리
        interest_css = "MainInfo_rate__em52j"
        interest_list = driver.find_elements(By.CLASS_NAME, interest_css)

        # 최고 우대금리
        high_interest = interest_list[0].text.split()[-1]

        # 최저 금리
        interest = interest_list[1].text.split()[-1]

        # 보여지는 기본, 최저 우대금리
        tmp_df['기본금리_'] = [interest] * len_table_df
        tmp_df['최저우대금리_'] = [high_interest]*len_table_df

    # 상품유형
        Tag_css = "TagList_tag__xQBbK"
        Tag_list = driver.find_elements(By.CLASS_NAME, Tag_css)
        Tag = ",".join([css.text for css in Tag_list])
        tmp_df['상품유형'] = [Tag] * len_table_df

        # 더보기
        more_css = "#PRODUCT_GUIDE > div > button"
        try:
            more_btn = driver.find_element(By.CSS_SELECTOR, more_css)
            more_btn.click()
        except:
            pass
        
        time.sleep(1)

        # item_info 일단 모으기
        item_info_name_css = "TextList_label__MxF08"
        
        item_info_name_list = driver.find_elements(By.CLASS_NAME,item_info_name_css )

        item_info_detail_css = "TextList_description__xhuFz"
        item_info_detail_list = driver.find_elements(By.CLASS_NAME, item_info_detail_css)

        for i in range(len(item_info_name_list)):
            if item_info_name_list[i].text =='기간':
                tmp_df['기간_'] = [item_info_detail_list[i].text]* len_table_df
                continue
            tmp_df[item_info_name_list[i].text] = [item_info_detail_list[i].text]* len_table_df
        
        
        # table 합치기
        if len(table_df) != 0:
            tmp_df = pd.concat([tmp_df, table_df], axis = 1)

        #만들어진 창을 닫고 프레임 전환
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        
        try:
            all_df_cma = pd.concat([all_df_cma, tmp_df], axis = 0)
        except:
            all_df_cma.to_csv("./tmp_saving.csv")
            #페이지
    next_page_css = "#content > div.ProductListSection_article__kDyT3 > div.Pagination_article__Rg8y3 > div > button.Pagination_button__ukte8.Pagination_next__2yfET"

    try:
        next_page_btn = driver.find_element(By.CSS_SELECTOR, next_page_css)
        next_page_btn.send_keys()
        next_page_btn.click()
        print(num_page)
        num_page += 1
        time.sleep(1)
    except:
        all_df_cma.to_csv("./cma.csv", encoding= 'utf-8-sig')
        driver.close()
        break
    

IndexError: list index out of range